In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import  LogisticRegression

In [2]:
# Load train and test data
train_data = pd.read_csv('final_proj_data.csv')
test_data = pd.read_csv('final_proj_test.csv')

In [3]:
# example for test_prediction
sub = pd.read_csv('final_proj_sample_submission.csv')
sub

,index,y
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2495,2495,0
2496,2496,0
2497,2497,0
2498,2498,0


In [4]:
# Train data
train_data

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,y
0,NaN,NaN,NaN,NaN,NaN,812.0,14.0,NaN,NaN,NaN,...,catzS2D,jySVZNlOJy,NaN,xG3x,Aoh3,ZI9m,ib5G6X1eUxUn6,mj86,NaN,0
1,NaN,NaN,NaN,NaN,NaN,2688.0,7.0,NaN,NaN,NaN,...,i06ocsg,LM8l689qOp,NaN,kG3k,WqMG,RAYp,55YFVY9,mj86,NaN,0
2,NaN,NaN,NaN,NaN,NaN,1015.0,14.0,NaN,NaN,NaN,...,P6pu4Vl,LM8l689qOp,NaN,kG3k,Aoh3,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN,0
3,NaN,NaN,NaN,NaN,NaN,168.0,0.0,NaN,NaN,NaN,...,BNrD3Yd,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,14.0,0.0,NaN,NaN,NaN,...,3B1QowC,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,...,CE7uk3u,jySVZNlOJy,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,0
9996,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,...,UpJuOS_,LM8l689qOp,NaN,NaN,TNEC,6fzt,F2FcTt7IdMT_v,NaN,NaN,0
9997,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,Ff09Jxo,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,0
9998,NaN,NaN,NaN,NaN,NaN,1267.0,14.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,kG3k,453m,ZI9m,TCU50_Yjmm6GIBZ0lL_,am7c,NaN,0


In [5]:
train_data.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var209,Var230,y
count,133.000000,266.0,266.000000,280.000000,2.410000e+02,8980.000000,8995.000000,0.0,133.000000,2.410000e+02,...,266.000000,0.0,133.000000,133.000000,266.000000,4206.000000,43.000000,0.0,0.0,10000.00000
mean,14.977444,0.0,341.052632,0.096429,2.338101e+05,1340.916258,6.860700,NaN,61.383459,3.672943e+05,...,6.180451,NaN,2.977444,20.601504,159.107368,272.455064,25725.112326,NaN,NaN,0.13050
std,66.456008,0.0,2810.606975,0.928243,5.532305e+05,2380.516758,6.300994,NaN,266.124849,8.234215e+05,...,12.177204,NaN,10.329764,93.736247,115.766972,86.752531,37487.484852,NaN,NaN,0.33687
min,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,...,0.000000,NaN,0.000000,0.000000,0.000000,6.000000,0.000000,NaN,NaN,0.00000
25%,0.000000,0.0,0.000000,0.000000,0.000000e+00,523.250000,0.000000,NaN,2.000000,0.000000e+00,...,0.000000,NaN,0.000000,0.000000,18.840000,204.000000,1312.875000,NaN,NaN,0.00000
50%,0.000000,0.0,0.000000,0.000000,0.000000e+00,861.000000,7.000000,NaN,18.000000,0.000000e+00,...,0.000000,NaN,0.000000,4.000000,194.670000,270.000000,10853.820000,NaN,NaN,0.00000
75%,16.000000,0.0,0.000000,0.000000,1.172350e+05,1428.000000,7.000000,NaN,40.000000,2.439360e+05,...,7.000000,NaN,0.000000,12.000000,247.080000,330.000000,37491.525000,NaN,NaN,0.00000
max,680.000000,0.0,42588.000000,9.000000,3.024000e+06,76195.000000,35.000000,NaN,2300.000000,6.394806e+06,...,64.000000,NaN,102.000000,878.000000,452.760000,642.000000,191167.200000,NaN,NaN,1.00000


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 231 entries, Var1 to y
dtypes: float64(191), int64(2), object(38)
memory usage: 17.6+ MB


In [7]:
train_data.isna().sum()

Var1       9867
Var2       9734
Var3       9734
Var4       9720
Var5       9759
          ...  
Var227        0
Var228        0
Var229     5561
Var230    10000
y             0
Length: 231, dtype: int64

In [8]:
train_data = train_data[train_data.columns[train_data.isna().mean().lt(0.35)]]

In [9]:
train_data.isna().sum()

Var6      1020
Var7      1005
Var13     1005
Var21     1020
Var22      920
          ... 
Var223    1013
Var226       0
Var227       0
Var228       0
y            0
Length: 68, dtype: int64

In [10]:
# Test data
test_data

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,ELof,rgKb,ZI9m,ib5G6X1eUxUn6,am7c,NaN
1,NaN,NaN,NaN,NaN,NaN,3192.0,28.0,NaN,NaN,NaN,...,oslk,QkgQQMs,LM8l689qOp,NaN,NaN,453m,RAYp,F2FyR07IdsN7I,am7c,NaN
2,NaN,NaN,NaN,NaN,NaN,756.0,0.0,NaN,NaN,NaN,...,oslk,bxCQb98,jySVZNlOJy,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,3892.0,21.0,NaN,NaN,NaN,...,oslk,0JeRt72,M_8D,NaN,ELof,Aoh3,RAYp,F2FyR07IdsN7I,mj86,NaN
4,NaN,NaN,NaN,NaN,NaN,672.0,7.0,NaN,NaN,NaN,...,oslk,0p8OTZB,LM8l689qOp,NaN,ELof,xb3V,RAYp,F2FyR07IdsN7I,mj86,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,NaN,NaN,NaN,NaN,NaN,700.0,7.0,NaN,NaN,NaN,...,oslk,Pbo79_c,NaN,NaN,NaN,7aLG,RAYp,F2FyR07IdsN7I,NaN,NaN
2496,NaN,NaN,NaN,NaN,NaN,1421.0,7.0,NaN,NaN,NaN,...,oslk,853QZmh,LM8l689qOp,NaN,ELof,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2497,NaN,NaN,NaN,NaN,NaN,1372.0,7.0,NaN,NaN,NaN,...,oslk,0p8OTZB,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2498,NaN,NaN,NaN,NaN,NaN,861.0,7.0,NaN,NaN,NaN,...,oslk,QuuQ5_S,M_8D,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


In [11]:
test_data.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var183,Var184,Var185,Var186,Var187,Var188,Var189,Var190,Var209,Var230
count,38.000000,78.0,78.000000,72.000000,6.100000e+01,2229.000000,2228.000000,0.0,38.000000,6.100000e+01,...,7.800000e+01,78.000000,0.0,38.000000,38.000000,78.000000,1077.000000,11.000000,0.0,0.0
mean,9.263158,0.0,820.461538,0.250000,2.876863e+05,1309.423957,6.871185,NaN,46.263158,2.860579e+05,...,9.876592e+04,5.743590,NaN,2.368421,19.263158,177.723846,270.239554,23779.930909,NaN,NaN
std,11.544706,0.0,3470.654157,1.489399,8.448970e+05,2655.285857,6.328531,NaN,66.534561,7.833744e+05,...,2.243013e+05,9.894818,NaN,6.322081,33.619239,116.309592,88.919402,33518.730600,NaN,NaN
min,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,NaN,0.000000,0.000000e+00,...,0.000000e+00,0.000000,NaN,0.000000,0.000000,0.000000,36.000000,456.030000,NaN,NaN
25%,0.000000,0.0,0.000000,0.000000,0.000000e+00,490.000000,0.000000,NaN,8.500000,0.000000e+00,...,0.000000e+00,0.000000,NaN,0.000000,2.000000,19.920000,204.000000,2079.630000,NaN,NaN
50%,8.000000,0.0,0.000000,0.000000,0.000000e+00,854.000000,7.000000,NaN,27.000000,0.000000e+00,...,0.000000e+00,0.000000,NaN,0.000000,9.000000,205.770000,270.000000,3529.350000,NaN,NaN
75%,16.000000,0.0,0.000000,0.000000,1.338100e+05,1435.000000,7.000000,NaN,55.500000,5.004000e+03,...,7.728100e+04,8.000000,NaN,0.000000,21.500000,272.325000,330.000000,30532.050000,NaN,NaN
max,40.000000,0.0,22461.000000,9.000000,4.397645e+06,91126.000000,35.000000,NaN,358.000000,4.665600e+06,...,1.335952e+06,48.000000,NaN,36.000000,154.000000,362.520000,642.000000,101984.400000,NaN,NaN


In [12]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 230 entries, Var1 to Var230
dtypes: float64(191), int64(1), object(38)
memory usage: 4.4+ MB


In [13]:
test_data.isna().sum()

Var1      2462
Var2      2422
Var3      2422
Var4      2428
Var5      2439
          ... 
Var226       0
Var227       0
Var228       0
Var229    1423
Var230    2500
Length: 230, dtype: int64

In [14]:
test_data = test_data[test_data.columns[test_data.isna().mean().lt(0.35)]]

In [15]:
test_data.isna().sum()

Var6      271
Var7      272
Var13     272
Var21     271
Var22     249
         ... 
Var222      0
Var223    251
Var226      0
Var227      0
Var228      0
Length: 67, dtype: int64

In [16]:
# Split for train and test
X_train = train_data.drop(columns=['y'])
y_train = train_data['y']
X_test = test_data

In [17]:
# categorical and numeric
cat_features = X_train.select_dtypes(include='object').columns
num_features = X_train.select_dtypes(include=np.number).columns

In [18]:
# Impute missing values in numeric features
num_imputer = SimpleImputer(strategy='mean')
X_train_num_imputed = num_imputer.fit_transform(X_train[num_features])
X_test_num_imputed = num_imputer.transform(X_test[num_features])

In [19]:
# Standardize numeric features
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num_imputed)
X_test_num_scaled = scaler.transform(X_test_num_imputed)

In [20]:
# Impute missing values in categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_cat_imputed = cat_imputer.fit_transform(X_train[cat_features])
X_test_cat_imputed = cat_imputer.transform(X_test[cat_features])

In [21]:
# Encode categorical features using TargetEncoder
encoder = TargetEncoder()
X_train_cat_encoded = encoder.fit_transform(X_train[cat_features], y_train)
X_test_cat_encoded = encoder.transform(X_test[cat_features])

In [22]:
# Combine numeric and categorical features
X_train_processed = np.hstack((X_train_num_scaled, X_train_cat_encoded))
X_test_processed = np.hstack((X_test_num_scaled, X_test_cat_encoded))

Use GridSearchCV for search best params <br>
Results: <br>
Best Parameters: {'learning_rate': 0.2, 'max_depth': 3, 'max_iter': 200, 'min_samples_leaf': 100} <br>
Best Balanced Accuracy: 0.83 

In [23]:
# from sklearn.model_selection import GridSearchCV

# # Define a parameter grid to search
# param_grid = {
#     'max_iter': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'min_samples_leaf': [20, 50, 100]
# }

# # Initialize the model
# base_model = HistGradientBoostingClassifier(random_state=42)

# # Set up the grid search
# grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, scoring='balanced_accuracy', cv=5)
# grid_search.fit(X_train_processed, y_train)

# # Print the best parameters and the best score
# print(f"Best Parameters: {grid_search.best_params_}")
# print(f"Best Balanced Accuracy: {grid_search.best_score_:.2f}")


# # Best Parameters: {'learning_rate': 0.2, 'max_depth': 3, 'max_iter': 200, 'min_samples_leaf': 100}
# # Best Balanced Accuracy: 0.83


In [24]:
# Learn model and make predictions
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', HistGradientBoostingClassifier(max_iter=200, learning_rate=0.2, max_depth=3, min_samples_leaf=100, random_state=42))
]

stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_model.fit(X_train_processed, y_train)
y_test_pred = stacking_model.predict(X_test_processed)



Mean Balanced Accuracy preview

In [28]:
# # Mean Balanced Accuracy preview
# from sklearn.model_selection import StratifiedKFold, cross_val_score

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# scores = cross_val_score(stacking_model, X_train_processed, y_train, cv=skf, scoring='balanced_accuracy')
# print(f"Mean Balanced Accuracy: {scores.mean():.4f}")

Mean Balanced Accuracy: 0.8079


In [26]:
# Create a DataFrame for predictions 
test_predictions = pd.DataFrame({
    'index' : sub['index'],
    'y': y_test_pred
})

In [27]:
# Save to CSV
test_predictions.to_csv('test_predictions.csv', index=False)